In [3]:
import dlib
import cv2
import numpy as np
import random
import face_recognition
from moviepy.editor import *
from moviepy.video.fx.all import *
import tracemalloc
tracemalloc.start()

cap = cv2.VideoCapture('./volleyball.mp4') 
fps = cap.get(cv2.CAP_PROP_FPS)
# 計算影片總隔數
total__frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#ms = cap.get(cv2.CAP_PROP_POS_MSEC)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))    #1280    # 取得畫面尺寸
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))   #720
font = 2;    lt = 10
print(fps,w,h,total__frames)

# 設定時間格數
start_frame = 280
end_frame = 560
end_frame_invert = 640
end_frame_RGB2BGR = 840
end_frame_2in1 = 1400

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# 創建一個 OpenCV 的 VideoWriter 物件，這個物件可以用來將影像序列寫入到一個影片文件中
# 'XVID' 代表的是 Xvid 編碼器，它是一種常用的 MPEG-4 視頻編碼器

out = cv2.VideoWriter('openCV_HW.avi', fourcc, 30.0, (w, h))
total_frames = end_frame - start_frame
scale_step = 1.0 / (end_frame - start_frame)
# #旋轉速度
rotation_step = 4 * 360.0 / (end_frame - start_frame)

colormaps = [cv2.COLORMAP_AUTUMN, cv2.COLORMAP_BONE, cv2.COLORMAP_JET, cv2.COLORMAP_WINTER,
             cv2.COLORMAP_RAINBOW, cv2.COLORMAP_OCEAN, cv2.COLORMAP_SUMMER, cv2.COLORMAP_SPRING,
             cv2.COLORMAP_COOL, cv2.COLORMAP_HSV, cv2.COLORMAP_PINK, cv2.COLORMAP_HOT,
             cv2.COLORMAP_PARULA, cv2.COLORMAP_MAGMA, cv2.COLORMAP_INFERNO, cv2.COLORMAP_PLASMA]

def get_random_colormaps():#變換colormap順序
    colormaps = list(range(0, 16))
    random.shuffle(colormaps)
    return colormaps

colormap_counter = 0
colormaps = get_random_colormaps()

def adjust_gamma(image, gamma):
    invgamma =1/(1/gamma)
    table = np.array([((i / 255.0) ** invgamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

frames = [np.zeros((h//4, w//4, 3), dtype=np.uint8)] * 16 #處理邊界

# ==================================
# 設定影格以及效果
frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret or cv2.waitKey(5) == 27: break
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    frame_Original = frame

    if frame_number < start_frame:
        cv2.putText(frame, "1. Original", (10, 30), font, .8, (0,255,247), 2, lt)
    
    
    elif start_frame <= frame_number < end_frame:
        #scale = 1.0 - (frame_number - start_frame) * scale_step
        scale =  (frame_number - start_frame) * scale_step
        rotation = (frame_number - start_frame) * rotation_step
        h, w = frame.shape[:2]
        center = (w // 2, h // 2)
        # 計算縮放和旋轉矩陣
        transform_matrix = cv2.getRotationMatrix2D(center, rotation, scale)
        # 應用變換
        frame = cv2.warpAffine(frame, transform_matrix, (w, h))
        cv2.putText(frame, f"2. Rotation: 4 & Scale {scale*rotation:.0f} ", 
                (10, 30), font, .8, (0,0,255), 2, lt)
    
    elif end_frame <= frame_number < end_frame_invert:
        frame = cv2.bitwise_not(frame)
        
        cv2.putText(frame, "3. COLOR_INVERT", (10, 30), font, .8, (0, 255, 247), 2, lt)
        
    elif end_frame_invert <= frame_number < end_frame_RGB2BGR:
        # Convert the frame to grayscale
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        cv2.putText(frame, "4. Sobal ", (10, 30), font, .8, (0,255,247), 2, lt)
        
    elif end_frame_RGB2BGR <= frame_number < end_frame_2in1:
        new_width, new_height = w // 2, h
        # Resize the image to the new dimensions
        frame_resized = cv2.resize(frame, (new_width, new_height))
        # Create the flipped images
        frame3 = cv2.flip(frame_resized, 1)  # Flip horizontally

        # Place the images in the correct positions in img1
        img1 = np.zeros_like(frame_Original)
        img1[0:new_height, 0:new_width] = frame_resized
        img1[0:new_height, new_width:w] = frame3
        cv2.putText(img1, "5. 2 in 1 + Flip", (10, 30), font, .8, (0,255,247), 2, lt)
        frame = img1
    
    else:
        small_frame = cv2.resize(frame, (w//4, h//4))
        colormap_counter += 1
        if colormap_counter % 10 == 0:
            colormaps = get_random_colormaps()
        random_colormaps = random.sample(colormaps, len(colormaps))
        # Apply colormaps and create 4x4 grid
        for i in range(4):
            for j in range(4):
                frames[i*4+j] = cv2.applyColorMap(small_frame, colormaps[i*4+j])
        # Concatenate frames horizontally
        row1 = np.hstack(frames[:4])
        row2 = np.hstack(frames[4:8])
        row3 = np.hstack(frames[8:12])
        row4 = np.hstack(frames[12:])
        # Concatenate rows vertically
        result = np.vstack([row1, row2, row3, row4])
        cv2.putText(result, "6. 4x4 in 1 + Colormap", (10, 30), font, .8, (0, 255, 247), 2, lt)
        frame = result
    

    cv2.putText(frame, f'{cap.get(cv2.CAP_PROP_POS_FRAMES):.0f}/{total__frames} frames, FPS : {fps:.0f}',(10, 60), font, .8, (0,255,247), 2, lt)
    out.write(frame)
    cv2.imshow('frame',frame)
    frame_number += 1

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')
print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)

23.976023976023978 1280 720 1553
[ Top 10 ]
<__array_function__ internals>:5: size=8103 KiB, count=24, average=338 KiB
C:\Users\Tibame_EX14\AppData\Local\Temp\ipykernel_16676\2751191214.py:118: size=2702 KiB, count=41, average=65.9 KiB
C:\Users\Tibame_EX14\AppData\Local\Temp\ipykernel_16676\2751191214.py:62: size=2700 KiB, count=2, average=1350 KiB
C:\Users\Tibame_EX14\AppData\Local\Temp\ipykernel_16676\2751191214.py:100: size=1350 KiB, count=2, average=675 KiB
C:\Users\Tibame_EX14\AppData\Local\Temp\ipykernel_16676\2751191214.py:98: size=1350 KiB, count=2, average=675 KiB
C:\Users\Tibame_EX14\anaconda3\lib\selectors.py:315: size=288 KiB, count=9, average=32.0 KiB
C:\Users\Tibame_EX14\AppData\Local\Temp\ipykernel_16676\2751191214.py:110: size=169 KiB, count=2, average=84.4 KiB
C:\Users\Tibame_EX14\anaconda3\lib\site-packages\IPython\core\compilerop.py:101: size=156 KiB, count=2690, average=59 B
C:\Users\Tibame_EX14\anaconda3\lib\tokenize.py:528: size=98.0 KiB, count=1792, average=56 B
